In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [9]:
file_path = 'file:////home/talentum/spark/README.md'

# Create a baseRDD from the file path
baseRDD = sc.textFile(file_path)

# Split the lines of baseRDD into words
splitRDD = baseRDD.flatMap(lambda x: x.split(' '))
 
# Create a tuple of the word and 1 
splitRDD_map = splitRDD.map(lambda w: (w.strip(), 1))

# Count of the number of occurences of each word
resultRDD = splitRDD_map.reduceByKey(lambda x, y: x + y)

# Display the first 10 words and their frequencies
for word in resultRDD.take(10):
    print("{} --> {} counts". format(word[0], word[1]))

# --> 1 counts
Apache --> 1 counts
Spark --> 15 counts
 --> 72 counts
is --> 7 counts
a --> 9 counts
fast --> 1 counts
and --> 10 counts
general --> 3 counts
cluster --> 2 counts


Running this file in terminal 


```python
#!/usr/bin/python

#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext
file_path = 'file:////home/talentum/spark/README.md'

# Create a baseRDD from the file path
baseRDD = sc.textFile(file_path)

# Split the lines of baseRDD into words
splitRDD = baseRDD.flatMap(lambda x: x.split(' '))
 
# Create a tuple of the word and 1 
splitRDD_map = splitRDD.map(lambda w: (w.strip(), 1))

# Count of the number of occurences of each word
resultRDD = splitRDD_map.reduceByKey(lambda x, y: x + y)

# Display the first 10 words and their frequencies
for word in resultRDD.take(10):
    print("{} --> {} counts". format(word[0], word[1]))
```


```shell
talentum@talentum-virtual-machine:~$ which python
/usr/bin/python
talentum@talentum-virtual-machine:~$ gedit wordcount.py
talentum@talentum-virtual-machine:~$ spark-submit
Usage: spark-submit [options] <app jar | python file | R file> [app arguments]
Usage: spark-submit --kill [submission ID] --master [spark://...]
Usage: spark-submit --status [submission ID] --master [spark://...]
Usage: spark-submit run-example [options] example-class [example args]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
                              and executor classpaths.
  --packages                  Comma-separated list of maven coordinates of jars to include
                              on the driver and executor classpaths. Will search the local
                              maven repo, then maven central and any additional remote
                              repositories given by --repositories. The format for the
                              coordinates should be groupId:artifactId:version.
  --exclude-packages          Comma-separated list of groupId:artifactId, to exclude while
                              resolving the dependencies provided in --packages to avoid
                              dependency conflicts.
  --repositories              Comma-separated list of additional remote repositories to
                              search for the maven coordinates given with --packages.
  --py-files PY_FILES         Comma-separated list of .zip, .egg, or .py files to place
                              on the PYTHONPATH for Python apps.
  --files FILES               Comma-separated list of files to be placed in the working
                              directory of each executor. File paths of these files
                              in executors can be accessed via SparkFiles.get(fileName).

  --conf PROP=VALUE           Arbitrary Spark configuration property.
  --properties-file FILE      Path to a file from which to load extra properties. If not
                              specified, this will look for conf/spark-defaults.conf.

  --driver-memory MEM         Memory for driver (e.g. 1000M, 2G) (Default: 1024M).
  --driver-java-options       Extra Java options to pass to the driver.
  --driver-library-path       Extra library path entries to pass to the driver.
  --driver-class-path         Extra class path entries to pass to the driver. Note that
                              jars added with --jars are automatically included in the
                              classpath.

  --executor-memory MEM       Memory per executor (e.g. 1000M, 2G) (Default: 1G).

  --proxy-user NAME           User to impersonate when submitting the application.
                              This argument does not work with --principal / --keytab.

  --help, -h                  Show this help message and exit.
  --verbose, -v               Print additional debug output.
  --version,                  Print the version of current Spark.

 Cluster deploy mode only:
  --driver-cores NUM          Number of cores used by the driver, only in cluster mode
                              (Default: 1).

 Spark standalone or Mesos with cluster deploy mode only:
  --supervise                 If given, restarts the driver on failure.
  --kill SUBMISSION_ID        If given, kills the driver specified.
  --status SUBMISSION_ID      If given, requests the status of the driver specified.

 Spark standalone and Mesos only:
  --total-executor-cores NUM  Total cores for all executors.

 Spark standalone and YARN only:
  --executor-cores NUM        Number of cores per executor. (Default: 1 in YARN mode,
                              or all available cores on the worker in standalone mode)

 YARN-only:
  --queue QUEUE_NAME          The YARN queue to submit to (Default: "default").
  --num-executors NUM         Number of executors to launch (Default: 2).
                              If dynamic allocation is enabled, the initial number of
                              executors will be at least NUM.
  --archives ARCHIVES         Comma separated list of archives to be extracted into the
                              working directory of each executor.
  --principal PRINCIPAL       Principal to be used to login to KDC, while running on
                              secure HDFS.
  --keytab KEYTAB             The full path to the file that contains the keytab for the
                              principal specified above. This keytab will be copied to
                              the node running the Application Master via the Secure
                              Distributed Cache, for renewing the login tickets and the
                              delegation tokens periodically.
      
talentum@talentum-virtual-machine:~$ cat unset_jupyter.sh 
#!/bin/bash

unset PYSPARK_DRIVER_PYTHON
unset PYSPARK_DRIVER_PYTHON_OPTS
talentum@talentum-virtual-machine:~$ source unset_jupyter.sh 
talentum@talentum-virtual-machine:~$ spark-submit wordcount.py  
25/01/03 14:52:00 WARN util.Utils: Your hostname, talentum-virtual-machine resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s17)
25/01/03 14:52:00 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/01/03 14:52:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Traceback (most recent call last):
  File "/home/talentum/wordcount.py", line 5, in <module>
    baseRDD = sc.textFile(file_path)
NameError: name 'sc' is not defined
25/01/03 14:52:01 INFO util.ShutdownHookManager: Shutdown hook called
25/01/03 14:52:01 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-acccd12f-7d91-4298-981d-1f764b255e17
talentum@talentum-virtual-machine:~$ gedit wordcount.py
talentum@talentum-virtual-machine:~$ spark-submit wordcount.py  
25/01/03 14:54:38 WARN util.Utils: Your hostname, talentum-virtual-machine resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s17)
25/01/03 14:54:38 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/01/03 14:54:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/03 14:54:40 INFO spark.SparkContext: Running Spark version 2.4.5
25/01/03 14:54:40 INFO spark.SparkContext: Submitted application: Spark SQL basic example
25/01/03 14:54:40 INFO spark.SecurityManager: Changing view acls to: talentum
25/01/03 14:54:40 INFO spark.SecurityManager: Changing modify acls to: talentum
25/01/03 14:54:40 INFO spark.SecurityManager: Changing view acls groups to: 
25/01/03 14:54:40 INFO spark.SecurityManager: Changing modify acls groups to: 
25/01/03 14:54:40 INFO spark.SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(talentum); groups with view permissions: Set(); users  with modify permissions: Set(talentum); groups with modify permissions: Set()
25/01/03 14:54:40 INFO util.Utils: Successfully started service 'sparkDriver' on port 41245.
25/01/03 14:54:40 INFO spark.SparkEnv: Registering MapOutputTracker
25/01/03 14:54:40 INFO spark.SparkEnv: Registering BlockManagerMaster
25/01/03 14:54:40 INFO storage.BlockManagerMasterEndpoint: Using org.apache.spark.storage.DefaultTopologyMapper for getting topology information
25/01/03 14:54:40 INFO storage.BlockManagerMasterEndpoint: BlockManagerMasterEndpoint up
25/01/03 14:54:40 INFO storage.DiskBlockManager: Created local directory at /tmp/blockmgr-53f3af7e-d2da-4c09-b712-d11d648b39b0
25/01/03 14:54:40 INFO memory.MemoryStore: MemoryStore started with capacity 413.9 MB
25/01/03 14:54:40 INFO spark.SparkEnv: Registering OutputCommitCoordinator
25/01/03 14:54:41 INFO util.log: Logging initialized @3213ms
25/01/03 14:54:41 INFO server.Server: jetty-9.3.z-SNAPSHOT, build timestamp: unknown, git hash: unknown
25/01/03 14:54:41 INFO server.Server: Started @3409ms
25/01/03 14:54:41 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/03 14:54:41 INFO server.AbstractConnector: Started ServerConnector@28620b33{HTTP/1.1,[http/1.1]}{0.0.0.0:4041}
25/01/03 14:54:41 INFO util.Utils: Successfully started service 'SparkUI' on port 4041.
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@2b773671{/jobs,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@433385de{/jobs/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@27cc15d3{/jobs/job,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@37db06a6{/jobs/job/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@689978f6{/stages,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@2f3c8abb{/stages/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@2fdd2619{/stages/stage,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@288342ca{/stages/stage/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@6e28442d{/stages/pool,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@490835e1{/stages/pool/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@21eb6ada{/storage,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@134cf93f{/storage/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@3f69ce57{/storage/rdd,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@37c64366{/storage/rdd/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@4d8152fe{/environment,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@39f0df70{/environment/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@481aa56f{/executors,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@4d420279{/executors/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@a13af8d{/executors/threadDump,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@3527eae0{/executors/threadDump/json,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@288b10f6{/static,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@67a6fd2c{/,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@543277c{/api,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@7a43779f{/jobs/job/kill,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@79d7d614{/stages/stage/kill,null,AVAILABLE,@Spark}
25/01/03 14:54:41 INFO ui.SparkUI: Bound SparkUI to 0.0.0.0, and started at http://10.0.2.15:4041
25/01/03 14:54:41 INFO executor.Executor: Starting executor ID driver on host localhost
25/01/03 14:54:41 INFO util.Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 37963.
25/01/03 14:54:41 INFO netty.NettyBlockTransferService: Server created on 10.0.2.15:37963
25/01/03 14:54:41 INFO storage.BlockManager: Using org.apache.spark.storage.RandomBlockReplicationPolicy for block replication policy
25/01/03 14:54:41 INFO storage.BlockManagerMaster: Registering BlockManager BlockManagerId(driver, 10.0.2.15, 37963, None)
25/01/03 14:54:41 INFO storage.BlockManagerMasterEndpoint: Registering block manager 10.0.2.15:37963 with 413.9 MB RAM, BlockManagerId(driver, 10.0.2.15, 37963, None)
25/01/03 14:54:41 INFO storage.BlockManagerMaster: Registered BlockManager BlockManagerId(driver, 10.0.2.15, 37963, None)
25/01/03 14:54:41 INFO storage.BlockManager: Initialized BlockManager: BlockManagerId(driver, 10.0.2.15, 37963, None)
25/01/03 14:54:42 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@3ac69269{/metrics/json,null,AVAILABLE,@Spark}
25/01/03 14:54:42 INFO internal.SharedState: loading hive config file: file:/home/talentum/spark/conf/hive-site.xml
25/01/03 14:54:42 INFO internal.SharedState: spark.sql.warehouse.dir is not set, but hive.metastore.warehouse.dir is set. Setting spark.sql.warehouse.dir to the value of hive.metastore.warehouse.dir ('/user/hive/warehouse').
25/01/03 14:54:42 INFO internal.SharedState: Warehouse path is '/user/hive/warehouse'.
25/01/03 14:54:42 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@67ee1d2d{/SQL,null,AVAILABLE,@Spark}
25/01/03 14:54:42 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@53cea9a9{/SQL/json,null,AVAILABLE,@Spark}
25/01/03 14:54:42 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@7a3211eb{/SQL/execution,null,AVAILABLE,@Spark}
25/01/03 14:54:42 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@4cd733bf{/SQL/execution/json,null,AVAILABLE,@Spark}
25/01/03 14:54:42 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@234e3110{/static/sql,null,AVAILABLE,@Spark}
25/01/03 14:54:43 INFO state.StateStoreCoordinatorRef: Registered StateStoreCoordinator endpoint
25/01/03 14:54:44 INFO memory.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 243.2 KB, free 413.7 MB)
25/01/03 14:54:44 INFO memory.MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 23.6 KB, free 413.7 MB)
25/01/03 14:54:44 INFO storage.BlockManagerInfo: Added broadcast_0_piece0 in memory on 10.0.2.15:37963 (size: 23.6 KB, free: 413.9 MB)
25/01/03 14:54:44 INFO spark.SparkContext: Created broadcast 0 from textFile at NativeMethodAccessorImpl.java:0
25/01/03 14:54:44 INFO mapred.FileInputFormat: Total input paths to process : 1
25/01/03 14:54:45 INFO spark.SparkContext: Starting job: runJob at PythonRDD.scala:153
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Registering RDD 3 (reduceByKey at /home/talentum/wordcount.py:25) as input to shuffle 0
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Got job 0 (runJob at PythonRDD.scala:153) with 1 output partitions
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Final stage: ResultStage 1 (runJob at PythonRDD.scala:153)
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Parents of final stage: List(ShuffleMapStage 0)
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Missing parents: List(ShuffleMapStage 0)
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Submitting ShuffleMapStage 0 (PairwiseRDD[3] at reduceByKey at /home/talentum/wordcount.py:25), which has no missing parents
25/01/03 14:54:45 INFO memory.MemoryStore: Block broadcast_1 stored as values in memory (estimated size 11.2 KB, free 413.7 MB)
25/01/03 14:54:45 INFO memory.MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 7.3 KB, free 413.6 MB)
25/01/03 14:54:45 INFO storage.BlockManagerInfo: Added broadcast_1_piece0 in memory on 10.0.2.15:37963 (size: 7.3 KB, free: 413.9 MB)
25/01/03 14:54:45 INFO spark.SparkContext: Created broadcast 1 from broadcast at DAGScheduler.scala:1163
25/01/03 14:54:45 INFO scheduler.DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 0 (PairwiseRDD[3] at reduceByKey at /home/talentum/wordcount.py:25) (first 15 tasks are for partitions Vector(0))
25/01/03 14:54:45 INFO scheduler.TaskSchedulerImpl: Adding task set 0.0 with 1 tasks
25/01/03 14:54:45 INFO scheduler.TaskSetManager: Starting task 0.0 in stage 0.0 (TID 0, localhost, executor driver, partition 0, PROCESS_LOCAL, 7884 bytes)
25/01/03 14:54:45 INFO executor.Executor: Running task 0.0 in stage 0.0 (TID 0)
25/01/03 14:54:45 INFO rdd.HadoopRDD: Input split: file:/home/talentum/spark/README.md:0+3756
25/01/03 14:54:46 INFO python.PythonRunner: Times: total = 437, boot = 414, init = 19, finish = 4
25/01/03 14:54:46 INFO executor.Executor: Finished task 0.0 in stage 0.0 (TID 0). 1758 bytes result sent to driver
25/01/03 14:54:46 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 1020 ms on localhost (executor driver) (1/1)
25/01/03 14:54:46 INFO scheduler.TaskSchedulerImpl: Removed TaskSet 0.0, whose tasks have all completed, from pool 
25/01/03 14:54:46 INFO python.PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 56841
25/01/03 14:54:46 INFO scheduler.DAGScheduler: ShuffleMapStage 0 (reduceByKey at /home/talentum/wordcount.py:25) finished in 1.317 s
25/01/03 14:54:46 INFO scheduler.DAGScheduler: looking for newly runnable stages
25/01/03 14:54:46 INFO scheduler.DAGScheduler: running: Set()
25/01/03 14:54:46 INFO scheduler.DAGScheduler: waiting: Set(ResultStage 1)
25/01/03 14:54:46 INFO scheduler.DAGScheduler: failed: Set()
25/01/03 14:54:46 INFO scheduler.DAGScheduler: Submitting ResultStage 1 (PythonRDD[6] at RDD at PythonRDD.scala:53), which has no missing parents
25/01/03 14:54:46 INFO memory.MemoryStore: Block broadcast_2 stored as values in memory (estimated size 7.9 KB, free 413.6 MB)
25/01/03 14:54:46 INFO memory.MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 5.1 KB, free 413.6 MB)
25/01/03 14:54:46 INFO storage.BlockManagerInfo: Added broadcast_2_piece0 in memory on 10.0.2.15:37963 (size: 5.1 KB, free: 413.9 MB)
25/01/03 14:54:46 INFO spark.SparkContext: Created broadcast 2 from broadcast at DAGScheduler.scala:1163
25/01/03 14:54:46 INFO scheduler.DAGScheduler: Submitting 1 missing tasks from ResultStage 1 (PythonRDD[6] at RDD at PythonRDD.scala:53) (first 15 tasks are for partitions Vector(0))
25/01/03 14:54:46 INFO scheduler.TaskSchedulerImpl: Adding task set 1.0 with 1 tasks
25/01/03 14:54:46 INFO scheduler.TaskSetManager: Starting task 0.0 in stage 1.0 (TID 1, localhost, executor driver, partition 0, ANY, 7662 bytes)
25/01/03 14:54:46 INFO executor.Executor: Running task 0.0 in stage 1.0 (TID 1)
25/01/03 14:54:46 INFO storage.ShuffleBlockFetcherIterator: Getting 1 non-empty blocks including 1 local blocks and 0 remote blocks
25/01/03 14:54:46 INFO storage.ShuffleBlockFetcherIterator: Started 0 remote fetches in 8 ms
25/01/03 14:54:46 INFO python.PythonRunner: Times: total = 36, boot = -643, init = 678, finish = 1
25/01/03 14:54:46 INFO executor.Executor: Finished task 0.0 in stage 1.0 (TID 1). 1930 bytes result sent to driver
25/01/03 14:54:46 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 136 ms on localhost (executor driver) (1/1)
25/01/03 14:54:46 INFO scheduler.DAGScheduler: ResultStage 1 (runJob at PythonRDD.scala:153) finished in 0.191 s
25/01/03 14:54:46 INFO scheduler.TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
25/01/03 14:54:46 INFO scheduler.DAGScheduler: Job 0 finished: runJob at PythonRDD.scala:153, took 1.786293 s
# --> 1 counts
Apache --> 1 counts
Spark --> 15 counts
 --> 72 counts
is --> 7 counts
a --> 9 counts
fast --> 1 counts
and --> 10 counts
general --> 3 counts
cluster --> 2 counts
25/01/03 14:54:46 INFO spark.SparkContext: Invoking stop() from shutdown hook
25/01/03 14:54:46 INFO server.AbstractConnector: Stopped Spark@28620b33{HTTP/1.1,[http/1.1]}{0.0.0.0:4041}
25/01/03 14:54:46 INFO ui.SparkUI: Stopped Spark web UI at http://10.0.2.15:4041
25/01/03 14:54:46 INFO spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
25/01/03 14:54:46 INFO memory.MemoryStore: MemoryStore cleared
25/01/03 14:54:46 INFO storage.BlockManager: BlockManager stopped
25/01/03 14:54:47 INFO storage.BlockManagerMaster: BlockManagerMaster stopped
25/01/03 14:54:47 INFO scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
25/01/03 14:54:47 INFO spark.SparkContext: Successfully stopped SparkContext
25/01/03 14:54:47 INFO util.ShutdownHookManager: Shutdown hook called
25/01/03 14:54:47 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-8bdaaade-d68d-4b50-80e3-f25a5e9bca24/pyspark-7ff7281c-2513-4d18-acfd-755c3191caa3
25/01/03 14:54:47 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-d9898e92-5160-46f8-9c21-196b24c2b74f
25/01/03 14:54:47 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-8bdaaade-d68d-4b50-80e3-f25a5e9bca24

25/01/03 14:54:46 INFO server.AbstractConnector: Stopped Spark@28620b33{HTTP/1.1,[http/1.1]}{0.0.0.0:4041}
25/01/03 14:54:46 INFO ui.SparkUI: Stopped Spark web UI at http://10.0.2.15:4041
25/01/03 14:54:46 INFO spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
25/01/03 14:54:46 INFO memory.MemoryStore: MemoryStore cleared
25/01/03 14:54:46 INFO storage.BlockManager: BlockManager stopped
25/01/03 14:54:47 INFO storage.BlockManagerMaster: BlockManagerMaster stopped
25/01/03 14:54:47 INFO scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
25/01/03 14:54:47 INFO spark.SparkContext: Successfully stopped SparkContext
25/01/03 14:54:47 INFO util.ShutdownHookManager: Shutdown hook called
25/01/03 14:54:47 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-8bdaaade-d68d-4b50-80e3-f25a5e9bca24/pyspark-7ff7281c-2513-4d18-acfd-755c3191caa3
25/01/03 14:54:47 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-d9898e92-5160-46f8-9c21-196b24c2b74f
25/01/03 14:54:47 INFO util.ShutdownHookManager: Deleting directory /tmp/spark-8bdaaade-d68d-4b50-80e3-f25a5e9bca24
talentum@talentum-virtual-machine:~$ cd conf
bash: cd: conf: No such file or directory
talentum@talentum-virtual-machine:~$ cd spark/conf/
talentum@talentum-virtual-machine:~/spark/conf$ ls -lh
total 48K
-rw-r--r-- 1 talentum talentum  996 Feb  3  2020 docker.properties.template
-rw-r--r-- 1 talentum talentum 1.1K Feb  3  2020 fairscheduler.xml.template
lrwxrwxrwx 1 talentum talentum   38 Oct 29  2021 hive-site.xml -> /home/talentum/hive/conf/hive-site.xml
-rw-r--r-- 1 talentum talentum 2.0K Feb  3  2020 log4j.properties.template
-rw-r--r-- 1 talentum talentum 7.7K Feb  3  2020 metrics.properties.template
-rw-r--r-- 1 talentum talentum  865 Feb  3  2020 slaves.template
-rw-r--r-- 1 talentum talentum 1.3K Nov  4  2020 spark-defaults.conf
-rw-r--r-- 1 talentum talentum 1.3K Feb  3  2020 spark-defaults.conf.template
-rwxr-xr-x 1 talentum talentum 4.2K Nov  4  2020 spark-env.sh
-rwxr-xr-x 1 talentum talentum 4.2K Feb  3  2020 spark-env.sh.template
talentum@talentum-virtual-machine:~/spark/conf$ cp log4j.properties.template log4j.properties 
talentum@talentum-virtual-machine:~/spark/conf$ gedit log4j.properties    # change level to WARN
talentum@talentum-virtual-machine:~/spark/conf$ cd ../../
talentum@talentum-virtual-machine:~$ spark-submit wordcount.py  
25/01/03 14:59:02 WARN Utils: Your hostname, talentum-virtual-machine resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s17)
25/01/03 14:59:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/01/03 14:59:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/03 14:59:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
# --> 1 counts
Apache --> 1 counts
Spark --> 15 counts
 --> 72 counts
is --> 7 counts
a --> 9 counts
fast --> 1 counts
and --> 10 counts
general --> 3 counts
cluster --> 2 counts 
```